In [ ]:
%load_ext autoreload
%autoreload 2

# Cross Validation Functions

> Fill in a module description here

In [ ]:
#| default_exp crossValidation

In [ ]:
#| hide
from nbdev.showdoc import *
# from nbdev.qmd import *

In [ ]:
#| export
from __future__ import annotations
from fastcore.docments import *
from fastcore.test import *
from fastcore.utils import *

import pandas as pd
import numpy as np

from joblib import Parallel, delayed, dump, load
from collections import defaultdict
import copy

from sklearn.model_selection import ParameterGrid, ParameterSampler
from sklearn.base import clone

from dddex.baseClasses import BaseLSx
from dddex.wSAA import SampleAverageApproximation

## CV - Pinball Loss

In [ ]:
#| export

class QuantileCrossValidation:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 # An object with a `predict` method that must (!) have an argument called `probs`
                 # that specifies which quantiles to predict. Further, `estimator` needs
                 # a `set_params` and `fit` method.
                 estimator, 
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGrid: dict, 
                 randomSearch: bool=False, # Whether to use randomized search or grid search
                 # Number of parameter settings that are sampled if `randomSearch == True`. 
                 # n_iter trades off runtime vs quality of the solution.
                 nIter: int=None,
                 # iterable of floats between 0 and 1. These determine the p-quantiles being predicted 
                 # to evaluate performance of each parameter setting.
                 probs: list=[i / 100 for i in range(1, 100, 1)],
                 # If True, for each p-quantile a fitted LSF with best binSize to predict it is returned. 
                 # Otherwise only one LSF is returned that is best over all probs.
                 refitPerProb: bool=False, 
                 n_jobs: int=None, # Number of jobs to run in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None, 
                 ):
        
        # CHECKS  
        # if not isinstance(estimatorLSx, (BaseLSx)):
        #     raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")               
        
        if np.any(np.array(probs) > 1) or np.any(np.array(probs) < 0): 
            raise ValueError("probs must only contain numbers between 0 and 1!") 
        
        #---
        
        if randomSearch:
            self.parameterGrid = list(ParameterSampler(param_distributions = parameterGrid,
                                                       n_iter = nIter,
                                                       random_state = random_state).__iter__())
            
            self.randomSearch = True
            self.nIter = nIter
            self.random_state = random_state
            
        else:
            self.parameterGrid = ParameterGrid(parameterGrid)
            self.randomSearch = False
            
        #---
        
        self.estimator = copy.deepcopy(estimator)
        self.cvFolds = cvFolds
        self.probs = probs
        self.refitPerProb = refitPerProb
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestParams_perProb = None
        self.bestEstimator = None
        self.bestEstimator_perProb = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: QuantileCrossValidation, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScore)(estimator = copy.deepcopy(self.estimator),
                                                                         parameterGrid = self.parameterGrid,
                                                                         cvFold = cvFold,
                                                                         probs = self.probs,
                                                                         X = X,
                                                                         y = y) for cvFold in self.cvFolds)
    
    # scoresPerFold = [getFoldScore(estimator = copy.deepcopy(self.estimator),
    #                               parameterGrid = self.parameterGrid,
    #                               cvFold = cvFold,
    #                               probs = self.probs,
    #                               y = y,
    #                               X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    self.bestParams = paramsBest
    
    #---
    
    # BEST PARAMETER SETTINGS PER PROB
    paramsBestPerProbSeries = meanCostsDf.idxmin(axis = 0)
    paramsBestPerProb = {prob: dict(zip(meanCostsDf.index.names, paramsBestPerProbSeries.loc[prob])) for prob in self.probs}
    self.bestParams_perProb = paramsBestPerProb
    
    #---
    
    # REFITTING ESTIMATORS
    if self.refitPerProb:      
        
        # GET UNIQUE PARAMETER COMBS OF LSx THAT ARE BEST FOR ANY PROB
        paramsUnique = pd.DataFrame(paramsBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        estimatorDict = dict()
        
        for params in paramsUnique:
            estimator = copy.deepcopy(self.estimator)
            estimator.set_params(**params)
            
            estimator.fit(X = X, y = y)
            estimatorDict[tuple(params.values())] = estimator
        
        #---
        
        # DICTIONARY OF THE BEST LSx-ESTIMATORS PER PROB
        bestEstimatorPerProb = {prob: estimatorDict[tuple(paramsBestPerProb[prob].values())] for prob in self.probs}
        self.bestEstimator_perProb = bestEstimatorPerProb
        
    #---
    
    estimator = copy.deepcopy(self.estimator)
    estimator.set_params(**paramsBest)
    estimator.fit(X = X, y = y)

    self.bestEstimator = estimator

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScore(estimator, parameterGrid, cvFold, probs, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    quantilesDfSAA = SAA_fold.predict(X = XTestFold, probs = probs)
    
    costsDictSAA = {prob: getPinballLoss(decisions = quantilesDfSAA.loc[:, prob],
                                         yTest = yTestFold,
                                         prob = prob) for prob in probs}
    #---
    
    # Necessary to ensure compatability with wSAA-models etc.
    try:
        estimator.refitPointEstimator(X = XTrainFold, 
                                      y = yTrainFold)
    except:
        pass

    costsPerParam = defaultdict(dict)

    for params in parameterGrid:

        estimator.set_params(**params)

        estimator.fit(X = XTrainFold,
                      y = yTrainFold)

        quantilesDf = estimator.predict(X = XTestFold,
                                        probs = probs)
        
        costsDict = {prob: getPinballLoss(decisions = quantilesDf.loc[:, prob],
                                          yTest = yTestFold,
                                          prob = prob) for prob in probs}
        
        # We have to capture the special case of costSAA == 0, because then we can't compute the 
        # Cost-Ratio using the actual definition.
        costsRatioDict = dict()
        for prob in probs:
            
            if costsDictSAA[prob] > 0:
                costRatio = costsDict[prob] / costsDictSAA[prob]
            else:
                if costDict[prob] == 0:
                    costRatio = 0
                else:
                    costRatio = 1

            costsRatioDict[prob] = costRatio

        costsPerParam[tuple(params.values())] = costsRatioDict

    #---

    costsDf = pd.DataFrame.from_dict(costsPerParam, orient = 'index')
    costsDf.index.names = list(params.keys())
    
    return costsDf

## CV - Combined - Pinball Loss

In [ ]:
#| export

class QuantileCrossValidationLSx:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 estimatorLSx, # A Level-Set based model.
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with LSx parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridLSx: dict,
                 # dict or list of dicts with parameters names (`str`) of the point predictor as keys
                 # and distributions or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridEstimator: dict,
                 randomSearchLSx: bool=False, # Whether to use randomized search or grid search for the LSx parameters.
                 # Whether to use randomized search or grid search for the point predictor parameters.
                 randomSearchEstimator: bool=False, 
                 # Number of parameter settings of the LSx model that are sampled if `randomSearchLSx == True`. 
                 # LSx parameter settings are usually relatively cheap to evaluate, so all sampled LSx paramater settings
                 # are evaluated for each point predictor parameter setting.
                 nIterLSx: int=None,
                 # Number of parameter settings of the underlying point predictor that are sampled if 
                 # `randomSearchEstimator == True`. nIterEstimator trades off runtime vs quality of the solution.
                 nIterEstimator: int=None,
                 # iterable of floats between 0 and 1. These determine the p-quantiles being predicted 
                 # to evaluate performance of each parameter setting.
                 probs: list=[i / 100 for i in range(1, 100, 1)], 
                 # If True, for each p-quantile a fitted LSF with best binSize to predict it is returned. 
                 # Otherwise only one LSF is returned that is best over all probs.
                 refitPerProb: bool=False, 
                 n_jobs: int=None, # number of folds being computed in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None,
                 ):
        
        # CHECKS  
        if not isinstance(estimatorLSx, (BaseLSx)):
            raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")
            
        if len(parameterGridEstimator) == 0:
            raise ValueError("No parameter candidates have been specified for the point predictor. If you want to only evaluate"
                             "parameter settings of the LSx-estimator itself, use the class `QuantileCrossValidation` instead or"
                             "provide a fixed parameter setting for the point predictor via `parameterGridEstimator`.")
            
        if len(parameterGridLSx) == 0:
            raise ValueError("No parameter candidates have been specified for the LSx model! If you want to only evaluate"
                             "parameter settings of the point predictor, use standard cross-validation classes or instead"
                             "provide a fixed parameter setting for the LS model via `parameterGridLSx`.")
            
        if np.any(np.array(probs) > 1) or np.any(np.array(probs) < 0): 
            raise ValueError("`probs` must only contain numbers between 0 and 1!")
            
        if len(probs) == 0:
            raise ValueError("`probs` must be specified!")
        
        #---
        
        if randomSearchLSx:
            self.parameterGridLSx = list(ParameterSampler(param_distributions = parameterGridLSx,
                                                          n_iter = nIterLSx,
                                                          random_state = random_state).__iter__())
            self.randomSearchLSx = True
            self.nIterLsx = nIterLSx
            self.random_state = random_state
        
        else:
            self.parameterGridLSx = ParameterGrid(parameterGridLSx)
            self.randomSearchLSx = False
            
        
        if randomSearchEstimator:
            
            self.parameterGridEstimator = list(ParameterSampler(param_distributions = parameterGridEstimator,
                                                                n_iter = nIterEstimator,
                                                                random_state = random_state).__iter__())
            self.randomSearchEstimator = True
            self.nIterEstimator = nIterEstimator
            self.random_state = random_state
            
        else:
            self.parameterGridEstimator = ParameterGrid(parameterGridEstimator)
            self.randomSearchEstimator = False
            
        #---
        
        self.estimatorLSx = copy.deepcopy(estimatorLSx)
        
        self.cvFolds = cvFolds
        self.probs = probs
        self.refitPerProb = refitPerProb
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestParams_perProb = None
        self.bestEstimator = None
        self.bestEstimator_perProb = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: QuantileCrossValidationLSx, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScoreLSx)(estimatorLSx = copy.deepcopy(self.estimatorLSx),
                                                                            parameterGridLSx = self.parameterGridLSx, 
                                                                            parameterGridEstimator = self.parameterGridEstimator,
                                                                            cvFold = cvFold,
                                                                            probs = self.probs,
                                                                            y = y,
                                                                            X = X) for cvFold in self.cvFolds)
    
    # scoresPerFold = [getFoldScoreLSx(estimatorLSx = copy.deepcopy(self.estimatorLSx),
    #                                  parameterGridLSx = self.parameterGridLSx,
    #                                  parameterGridEstimator = self.parameterGridEstimator,
    #                                  cvFold = cvFold,
    #                                  probs = self.probs,
    #                                  y = y,
    #                                  X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    
    paramsLSxNames = self.estimatorLSx._get_param_names()
    paramsLSxBest = {paramName: value for paramName, value in paramsBest.items() if paramName in paramsLSxNames}
    paramsEstimatorBest = {paramName: value for paramName, value in paramsBest.items() if not paramName in paramsLSxNames}
    
    self.bestParams = paramsBest
    self.bestParamsLSx = paramsLSxBest
    self.bestParamsEstimator = paramsEstimatorBest
    
    #---
    
    # BEST PARAMETER SETTINGS PER PROB
    paramsBestPerProbSeries = meanCostsDf.idxmin(axis = 0)
    paramsBestPerProb = {prob: dict(zip(meanCostsDf.index.names, paramsBestPerProbSeries.loc[prob])) for prob in self.probs}
    
    paramsLSxBestPerProb = {prob: {paramName: value for paramName, value in paramsBestPerProb[prob].items() 
                                   if paramName in paramsLSxNames} for prob in self.probs}
    paramsEstimatorBestPerProb = {prob: {paramName: value for paramName, value in paramsBestPerProb[prob].items() 
                                   if not paramName in paramsLSxNames} for prob in self.probs}
    
    self.bestParams_perProb = paramsBestPerProb
    self.bestParamsLSx_perProb = paramsBestPerProb
    self.bestParamsEstimator_perProb = paramsBestPerProb
    
    #---
    
    # REFITTING ESTIMATORS
    if self.refitPerProb:
        
        # GET UNIQUE PARAMETER COMBS OF ESTIMATOR THAT ARE BEST FOR ANY PROB
        paramsEstimatorUnique = pd.DataFrame(paramsEstimatorBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        estimatorDict = dict()
        
        for params in paramsEstimatorUnique:
            estimator = clone(self.estimatorLSx.estimator)
            estimator.set_params(**params)
            estimator.fit(X = X, y = y)
            estimatorDict[tuple(params.values())] = estimator
        
        #---
        
        # GET UNIQUE PARAMETER COMBS OF LSx THAT ARE BEST FOR ANY PROB
        paramsUnique = pd.DataFrame(paramsBestPerProb.values()).drop_duplicates().to_dict(orient = 'records')
        
        estimatorLSxDict = dict()
        for params in paramsUnique:
            paramsLSx = {paramName: value for paramName, value in params.items() if paramName in paramsLSxNames}
            paramsEstimatorTuple = tuple(value for paramName, value in params.items() if not paramName in paramsLSxNames)
            
            estimator = estimatorDict[paramsEstimatorTuple]
            estimatorLSx = copy.deepcopy(self.estimatorLSx)
            
            estimatorLSx.set_params(**paramsLSx, 
                                    estimator = estimator)
            
            estimatorLSx.fit(X = X, y = y)
            estimatorLSxDict[tuple(params.values())] = estimatorLSx
        
        #---
        
        # DICTIONARY OF THE BEST LSx-ESTIMATORS PER PROB
        bestEstimatorPerProb = {prob: estimatorLSxDict[tuple(paramsBestPerProb[prob].values())] for prob in self.probs}
        self.bestEstimator_perProb = bestEstimatorPerProb
        
    #---
    
    estimatorLSx = copy.deepcopy(self.estimatorLSx)
    
    estimator = clone(estimatorLSx.estimator)
    estimator.set_params(**paramsEstimatorBest)
    estimator.fit(X = X, y = y)
    
    estimatorLSx.set_params(**paramsLSxBest,
                            estimator = estimator)
    estimatorLSx.fit(X = X, y = y)

    self.bestEstimator = estimatorLSx

In [ ]:
# show_doc(CrossValidationLSx_combined.fit)

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScoreLSx(estimatorLSx, parameterGridLSx, parameterGridEstimator, cvFold, probs, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    quantilesDfSAA = SAA_fold.predict(X = XTestFold, probs = probs)
    
    costsDictSAA = {prob: getPinballLoss(decisions = quantilesDfSAA.loc[:, prob],
                                         yTest = yTestFold,
                                         prob = prob) for prob in probs}
    
    #---
    
    costsDfList = list()
    
    for paramsEstimator in parameterGridEstimator:
        
        estimatorLSx.refitPointEstimator(X = XTrainFold, 
                                         y = yTrainFold,
                                         **paramsEstimator)

        #---

        costsPerParamLSx = defaultdict(dict)

        for paramsLSx in parameterGridLSx:

            estimatorLSx.set_params(**paramsLSx)

            estimatorLSx.fit(X = XTrainFold,
                             y = yTrainFold)

            quantilesDf = estimatorLSx.predict(X = XTestFold,
                                                 probs = probs)

            #---
            
            costsDict = {prob: getPinballLoss(decisions = quantilesDf.loc[:, prob],
                                              yTest = yTestFold,
                                              prob = prob) for prob in probs}
        
            # We have to capture the special case of costSAA == 0, because then we can't compute the 
            # Cost-Ratio using the actual definition.
            costsRatioDict = dict()
            for prob in probs:

                if costsDictSAA[prob] > 0:
                    costRatio = costsDict[prob] / costsDictSAA[prob]
                else:
                    if costDict[prob] == 0:
                        costRatio = 0
                    else:
                        costRatio = 1
                        
                costsRatioDict[prob] = costRatio
            
            costsPerParamLSx[tuple(paramsLSx.values())] = costsRatioDict

        #---
        
        costsDf = pd.DataFrame.from_dict(costsPerParamLSx, orient = 'index')
        
        paramsLSxNames = list(paramsLSx.keys())
        costsDf.index.names = paramsLSxNames

        costsDf = costsDf.reset_index(drop = False)
        for paramName, value in paramsEstimator.items():
            costsDf[paramName] = value

        paramNames = paramsLSxNames + list(paramsEstimator.keys())
        costsDf = costsDf.set_index(paramNames)
        
        costsDfList.append(costsDf)
        
    #---
    
    costsDf = pd.concat(costsDfList, axis = 0)
    
    return costsDf

## CV - Wasserstein Distance

In [ ]:
#| export

class DensityCrossValidation:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 # An object with a `predict` method that must (!) have an argument called `probs`
                 # that specifies which quantiles to predict. Further, `estimator` needs
                 # a `set_params` and `fit` method.
                 estimator, 
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGrid: dict, 
                 randomSearch: bool=False, # Whether to use randomized search or grid search
                 # Number of parameter settings that are sampled if `randomSearch == True`. 
                 # n_iter trades off runtime vs quality of the solution.
                 nIter: int=None,
                 p: int=1, # The order of the wasserstein distance to evaluate each hyperparameter settings.
                 n_jobs: int=None, # Number of jobs to run in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None, 
                 ):
        
        # CHECKS  
        # if not isinstance(estimatorLSx, (BaseLSx)):
        #     raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")               
        
        if not isinstance(p, int):
            raise ValueError("`p` must be an integer between 1 and inf!")
            
        #---
        
        if randomSearch:
            self.parameterGrid = list(ParameterSampler(param_distributions = parameterGrid,
                                                       n_iter = nIter,
                                                       random_state = random_state).__iter__())
            
            self.randomSearch = True
            self.nIter = nIter
            self.random_state = random_state
            
        else:
            self.parameterGrid = ParameterGrid(parameterGrid)
            self.randomSearch = False
            
        #---
        
        self.estimator = copy.deepcopy(estimator)
        self.cvFolds = cvFolds
        self.p = p
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestEstimator = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: DensityCrossValidation, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    # scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScore_wasserstein)(estimator = copy.deepcopy(self.estimator),
    #                                                                                  parameterGrid = self.parameterGrid,
    #                                                                                  cvFold = cvFold,
    #                                                                                  p = self.p,
    #                                                                                  X = X,
    #                                                                                  y = y) for cvFold in self.cvFolds)
    
    scoresPerFold = [getFoldScore_wasserstein(estimator = copy.deepcopy(self.estimator),
                                              parameterGrid = self.parameterGrid,
                                              cvFold = cvFold,
                                              p = self.p,
                                              y = y,
                                              X = X) for cvFold in self.cvFolds]
    
    # RESULTS
    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    # BEST PARAMETER SETTING
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    self.bestParams = paramsBest
    
    # REFITTING ESTIMATOR WITH BEST PARAMETERS
    estimator = copy.deepcopy(self.estimator)
    estimator.set_params(**paramsBest)
    estimator.fit(X = X, y = y)

    self.bestEstimator = estimator

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScore_wasserstein(estimator, parameterGrid, cvFold, p, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    densitiesSAA = SAA_fold.getWeights(X = XTestFold, outputType = 'onlyPositiveWeightsValues')
    
    wassersteinDistSAA = getWassersteinDistances(densities = densitiesSAA,
                                                 yTest = yTestFold,
                                                 p = p).sum()
    
    #---
    
    # Necessary to ensure compatability with wSAA-models etc.
    try:
        estimator.refitPointEstimator(X = XTrainFold, 
                                      y = yTrainFold)
    except:
        pass

    costsPerParam = defaultdict(dict)

    for params in parameterGrid:

        estimator.set_params(**params)

        estimator.fit(X = XTrainFold,
                      y = yTrainFold)
        
        #---
        
        densities = estimator.getWeights(X = XTestFold,
                                         outputType = 'onlyPositiveWeightsValues')
        
        wassersteinDist = getWassersteinDistances(densities = densities, 
                                                  yTest = yTestFold, 
                                                  p = p).sum()
        
        if wassersteinDistSAA > 0:
            wassersteinRatio = wassersteinDist / wassersteinDistSAA
        else:
            if wassersteinDist == 0:
                wassersteinRatio = 0
            else:
                wassersteinRatio = 1
                
        costsPerParam[tuple(params.values())] = {'wassersteinRatio': wassersteinRatio}

    #---
    # s = pd.Series(list(d.values()),index=pd.MultiIndex.from_tuples(d.keys()))
    costsDf = pd.DataFrame.from_dict(costsPerParam, orient = 'index')
    costsDf.index.names = list(params.keys())
    
    return costsDf

## CV - Combined - Wasserstein Distance

In [ ]:
#| export

class DensityCrossValidationLSx:
    """
    Class to efficiently tune the `binSize` parameter of all Level-Set based models.
    """
    
    def __init__(self,
                 estimatorLSx, # A Level-Set based model.
                 cvFolds, # An iterable yielding (train, test) splits as arrays of indices.
                 # dict or list of dicts with LSx parameters names (`str`) as keys and distributions
                 # or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridLSx: dict,
                 # dict or list of dicts with parameters names (`str`) of the point predictor as keys
                 # and distributions or lists of parameters to try. Distributions must provide a ``rvs``
                 # method for sampling (such as those from scipy.stats.distributions).
                 parameterGridEstimator: dict,
                 randomSearchLSx: bool=False, # Whether to use randomized search or grid search for the LSx parameters.
                 # Whether to use randomized search or grid search for the point predictor parameters.
                 randomSearchEstimator: bool=False, 
                 # Number of parameter settings of the LSx model that are sampled if `randomSearchLSx == True`. 
                 # LSx parameter settings are usually relatively cheap to evaluate, so all sampled LSx paramater settings
                 # are evaluated for each point predictor parameter setting.
                 nIterLSx: int=None,
                 # Number of parameter settings of the underlying point predictor that are sampled if 
                 # `randomSearchEstimator == True`. nIterEstimator trades off runtime vs quality of the solution.
                 nIterEstimator: int=None,
                 p: int=1, # The order of the wasserstein distance to evaluate each hyperparameter settings.
                 n_jobs: int=None, # number of folds being computed in parallel.
                 # Pseudo random number generator state used for random uniform sampling of parameter candidate values.
                 # Pass an int for reproducible output across multiple function calls.
                 random_state: int=None,
                 ):
        
        # CHECKS  
        if not isinstance(estimatorLSx, (BaseLSx)):
            raise ValueError("'estimatorLSx' has to be a 'LevelSetKDEx', 'LevelSetKDEx_NN_new' or a 'LevelSetKDEx_kNN' object!")
            
        if len(parameterGridEstimator) == 0:
            raise ValueError("No parameter candidates have been specified for the point predictor. If you want to only evaluate"
                             "parameter settings of the LSx-estimator itself, use the class `QuantileCrossValidation` instead or"
                             "provide a fixed parameter setting for the point predictor via `parameterGridEstimator`.")
            
        if len(parameterGridLSx) == 0:
            raise ValueError("No parameter candidates have been specified for the LSx model! If you want to only evaluate"
                             "parameter settings of the point predictor, use standard cross-validation classes or instead"
                             "provide a fixed parameter setting for the LS model via `parameterGridLSx`.")
            
        if not isinstance(p, int):
            raise ValueError("`p` must be an integer between 1 and inf!")
        
        #---
        
        if randomSearchLSx:
            self.parameterGridLSx = list(ParameterSampler(param_distributions = parameterGridLSx,
                                                          n_iter = nIterLSx,
                                                          random_state = random_state).__iter__())
            self.randomSearchLSx = True
            self.nIterLsx = nIterLSx
            self.random_state = random_state
        
        else:
            self.parameterGridLSx = ParameterGrid(parameterGridLSx)
            self.randomSearchLSx = False
            
        
        if randomSearchEstimator:
            
            self.parameterGridEstimator = list(ParameterSampler(param_distributions = parameterGridEstimator,
                                                                n_iter = nIterEstimator,
                                                                random_state = random_state).__iter__())
            self.randomSearchEstimator = True
            self.nIterEstimator = nIterEstimator
            self.random_state = random_state
            
        else:
            self.parameterGridEstimator = ParameterGrid(parameterGridEstimator)
            self.randomSearchEstimator = False
            
        #---
        
        self.estimatorLSx = copy.deepcopy(estimatorLSx)
        
        self.cvFolds = cvFolds
        self.p = p
        self.n_jobs = n_jobs
        
        self.bestParams = None
        self.bestEstimator = None
        self.cvResults = None
        self.cvResults_raw = None
        

In [ ]:
#| export

@patch
def fit(self: DensityCrossValidationLSx, 
        X: np.ndarray, # Feature matrix (has to work with the folds specified via `cvFolds`)
        y: np.ndarray, # Target values (has to work with the folds specified via `cvFolds`)
        ): 
    
    # Making sure that X and y are arrays to ensure correct subsetting via implicit indices.
    X = np.array(X)
    y = np.array(y)
    
    scoresPerFold = Parallel(n_jobs = self.n_jobs)(delayed(getFoldScoreLSx_wasserstein)(estimatorLSx = copy.deepcopy(self.estimatorLSx),
                                                                                        parameterGridLSx = self.parameterGridLSx, 
                                                                                        parameterGridEstimator = self.parameterGridEstimator,
                                                                                        cvFold = cvFold,
                                                                                        p = self.p,
                                                                                        y = y,
                                                                                        X = X) for cvFold in self.cvFolds)
    
    # scoresPerFold = [getFoldScoreLSx_wasserstein(estimatorLSx = copy.deepcopy(self.estimatorLSx),
    #                                              parameterGridLSx = self.parameterGridLSx,
    #                                              parameterGridEstimator = self.parameterGridEstimator,
    #                                              cvFold = cvFold,
    #                                              p = self.p,
    #                                              y = y,
    #                                              X = X) for cvFold in self.cvFolds]

    self.cvResults_raw = scoresPerFold
    meanCostsDf = sum(scoresPerFold) / len(scoresPerFold)
    self.cvResults = meanCostsDf
    
    #---
    
    # BEST PARAMETER SETTINGS OVER ALL PROBS
    meanCostsPerParam = meanCostsDf.mean(axis = 1)
    paramsBest = meanCostsPerParam.index[np.argmin(meanCostsPerParam)]
    paramsBest = dict(zip(meanCostsPerParam.index.names, paramsBest))
    
    paramsLSxNames = self.estimatorLSx._get_param_names()
    paramsLSxBest = {paramName: value for paramName, value in paramsBest.items() if paramName in paramsLSxNames}
    paramsEstimatorBest = {paramName: value for paramName, value in paramsBest.items() if not paramName in paramsLSxNames}
    
    self.bestParams = paramsBest
    self.bestParamsLSx = paramsLSxBest
    self.bestParamsEstimator = paramsEstimatorBest
        
    #---
    
    estimatorLSx = copy.deepcopy(self.estimatorLSx)
    
    estimator = clone(estimatorLSx.estimator)
    estimator.set_params(**paramsEstimatorBest)
    estimator.fit(X = X, y = y)
    
    estimatorLSx.set_params(**paramsLSxBest,
                            estimator = estimator)
    estimatorLSx.fit(X = X, y = y)

    self.bestEstimator = estimatorLSx

In [ ]:
# show_doc(CrossValidationLSx_combined.fit)

#### Scores for Single Fold

In [ ]:
#| export

# This function evaluates the newsvendor performance for different bin sizes for one specific fold.
# The considered bin sizes

def getFoldScoreLSx_wasserstein(estimatorLSx, parameterGridLSx, parameterGridEstimator, cvFold, p, X, y):
    
    indicesTrain = cvFold[0]
    indicesTest = cvFold[1]
    
    yTrainFold = y[indicesTrain]
    XTrainFold = X[indicesTrain]
    
    yTestFold = y[indicesTest]
    XTestFold = X[indicesTest]
    
    #---

    SAA_fold = SampleAverageApproximation()
    SAA_fold.fit(y = yTrainFold)

    densitiesSAA = SAA_fold.getWeights(X = XTestFold, outputType = 'onlyPositiveWeightsValues')
    
    wassersteinDistSAA = getWassersteinDistances(densities = densitiesSAA,
                                                 yTest = yTestFold,
                                                 p = p).sum()
    
    #---
    
    costsDfList = list()
    
    for paramsEstimator in parameterGridEstimator:
        
        estimatorLSx.refitPointEstimator(X = XTrainFold, 
                                         y = yTrainFold,
                                         **paramsEstimator)

        #---

        costsPerParamLSx = defaultdict(dict)

        for paramsLSx in parameterGridLSx:

            estimatorLSx.set_params(**paramsLSx)

            estimatorLSx.fit(X = XTrainFold,
                             y = yTrainFold)

            densities = estimatorLSx.getWeights(X = XTestFold,
                                                outputType = 'onlyPositiveWeightsValues')
        
            wassersteinDist = getWassersteinDistances(densities = densities, 
                                                      yTest = yTestFold, 
                                                      p = p).sum()

            if wassersteinDistSAA > 0:
                wassersteinRatio = wassersteinDist / wassersteinDistSAA
            else:
                if wassersteinDist == 0:
                    wassersteinRatio = 0
                else:
                    wassersteinRatio = 1

            costsPerParamLSx[tuple(paramsLSx.values())] = {'wassersteinRatio': wassersteinRatio}

        #---
        
        costsDf = pd.DataFrame.from_dict(costsPerParamLSx, orient = 'index')
        
        paramsLSxNames = list(paramsLSx.keys())
        costsDf.index.names = paramsLSxNames

        costsDf = costsDf.reset_index(drop = False)
        for paramName, value in paramsEstimator.items():
            costsDf[paramName] = value

        paramNames = paramsLSxNames + list(paramsEstimator.keys())
        costsDf = costsDf.set_index(paramNames)
        
        costsDfList.append(costsDf)
        
    #---
    
    costsDf = pd.concat(costsDfList, axis = 0)
    
    return costsDf

## Helper Functions

### Get Pinball Loss

In [ ]:
#| export

def getPinballLoss(decisions, yTest, prob):
    
    underageIndices = yTest > decisions
    
    underageCosts = prob * (yTest[underageIndices] - decisions[underageIndices]).sum()
    overageCosts = (1 - prob) * (decisions[~underageIndices] - yTest[~underageIndices]).sum()
    
    return underageCosts + overageCosts

### Get Wasserstein Distances

In [ ]:
#| export

def getWassersteinDistances(densities, yTest, p):
    
    wassersteinDists = list()
    
    for i in range(yTest.shape[0]):
        y = yTest[i]
        values = densities[i][1]
        probs = densities[i][0]

        if len(values.shape) == 1:
            values = values.reshape(-1, 1)
            y = y.reshape(-1, 1)

        wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**p, axis = 1))**(1/p))

    return np.array(wassersteinDists)

In [ ]:
#| hide

# def getWassersteinDistances(densities, yTest, p):
    
#     # CHECK IF TARGET VALUES ARE ONE- OR MULTI-DIMENSIONAL
#     # In the case of one-dimensional arrays, a faster algorithm that
#     # doesn't iterate over the test observations can be used.
#     if len(densities[0][1].shape) == 1:
        
#         lenList = list()
#         valuesList = list()
#         probsList = list()
        
#         for probs, values in densities:
#             lenList.append(values.shape[0])
#             probsList.append(probs)
#             valuesList.append(values)
            
#         lenMax = max(lenList)   
#         insertionCheck = np.arange(lenMax) < np.array(lenList)[:, None]

#         probsArray = np.zeros(shape = (len(probsList), lenMax))
#         valuesArray = np.zeros(shape = (len(valuesList), lenMax))

#         probsArray[insertionCheck] = np.concatenate(probsList, axis = 0)
#         valuesArray[insertionCheck] = np.concatenate(valuesList, axis = 0)
#         yTest = yTest.reshape(-1, 1)

#         wassersteinDists = np.sum(probsArray * np.abs(valuesArray - yTest)**p, axis = 1)**(1/p)
        
#         return wassersteinDists
    
#     #---
    
#     else:
#         wassersteinDists = list()
    
#         for i in range(yTest.shape[0]):
#             y = yTest[i]
#             values = densities[i][1]
#             probs = densities[i][0]

#             if len(values.shape) == 1:
#                 values = values.reshape(-1, 1)
#                 y = y.reshape(-1, 1)

#             wassersteinDists.append(np.sum(probs * np.sum(np.abs(values - y)**p, axis = 1))**(1/p))

#         return np.array(wassersteinDists)

### Grouped Time Series Split

In [ ]:
#| export

# This function creates the cross-validation folds for every time series. Usually you'd want all test-observations 
# of each fold to refer to the same time period, but this is impossible to ensure in the case of the two-step models,
# because the regression of the non-zero observations will always contain data of different time points. For that
# reason, we refrain from trying to ensure this consistency.
# Instead we organize our splits such that we always move a fixed amount of observations into the future from split
# to split for every time series. This fixed amount of observations is currently set to the test length of the
# corresponding time series.

# In case this function is supposed to be used in the two-step case, data simply has to be filtered before hand
# to only contain positive demand observations.

def groupedTimeSeriesSplit(data, kFolds, testLength, groupFeature, timeFeature):
    
    # We reset the index because we have to access 'group.index' later on and
    # want to make sure that we return the implicit numerical indices for our splits.
    data = data.reset_index(drop = True)

    dataGrouped = data.groupby(groupFeature)
    splitNumbers = np.flip(np.array(range(kFolds)))
    
    foldsList = list()

    for i in splitNumbers:

        trainIndicesList = list()
        valIndicesList = list()

        valIndicesDict = dict()

        for name, group in dataGrouped:

            timeMin = int(group[timeFeature].min())
            timeMax = int(group[timeFeature].max())

            validationTimeMax = timeMax - i * testLength
            trainTimeMax = validationTimeMax - testLength

            trainTimesGroup = np.array(range(timeMin, trainTimeMax + 1))
            valTimesGroup = np.array(range(trainTimeMax + 1, validationTimeMax + 1))

            trainIndicesCheck = [timePoint in trainTimesGroup for timePoint in group[timeFeature]]
            valIndicesCheck = [timePoint in valTimesGroup for timePoint in group[timeFeature]]

            trainIndicesGroup = group.index[trainIndicesCheck]
            valIndicesGroup = group.index[valIndicesCheck]

            trainIndicesList.append(trainIndicesGroup)
            valIndicesList.append(valIndicesGroup)

        trainIndices = np.concatenate(trainIndicesList)
        valIndices = np.concatenate(valIndicesList)
        fold = (trainIndices, valIndices)

        foldsList.append(fold)

    return foldsList

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

# Test Code

In [ ]:
# import numpy as np
# import pandas as pd

# from dddex.levelSetKDEx_univariate import *
# from dddex.loadData import *

# from lightgbm import LGBMRegressor
# from sklearn.model_selection import RandomizedSearchCV

# import time
# import ipdb

In [ ]:
# data, XTrain, yTrain, XTest, yTest = loadDataBakery()

In [ ]:
# LGBM = LGBMRegressor(n_jobs = 1)
# LGBM.fit(XTrain, yTrain)

In [ ]:
# dataTrain = data[data.label == 'train']

# cvFolds = groupedTimeSeriesSplit(data = dataTrain, 
#                                  kFolds = 3, 
#                                  testLength = 28, 
#                                  groupFeature = 'id', 
#                                  timeFeature = 'dayIndex')

In [ ]:
# paramGridEstimator = {'num_leaves': [20, 40, 60, 80, 100, 150],
#                       'max_depth': [-1, 3, 4, 5, 6, 7],
#                       'min_child_samples': [2, 4, 6, 8, 10, 13, 16, 20, 25, 50, 75, 100, 150, 200, 300, 400],
#                       'learning_rate': [0.05, 0.1, 0.15, 0.2],
#                       'n_estimators': [100, 200, 300, 400, 500, 600],
#                       'subsample': [0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1],
#                       'colsample_bytree': [0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1]}

# CVEstimator = RandomizedSearchCV(estimator = LGBM,
#                                  cv = cvFolds,
#                                  param_distributions = paramGridEstimator,
#                                  scoring = 'neg_mean_squared_error',
#                                  n_iter = 100,
#                                  refit = True,
#                                  n_jobs = len(cvFolds),
#                                  return_train_score = True,
#                                  verbose = 0)

# CVEstimator.fit(XTrain, yTrain)

In [ ]:
# # LSKDEx = LevelSetKDEx(estimator = CVEstimator.best_estimator_)
# LSKDEx = LevelSetKDEx(estimator = LGBM)

# paramGridLSx = {'binSize': [10, 20, 35, 50, 75, 100, 150, 200, 250, 400, 600, 800, 1000],
#                 'weightsByDistance': [True, False]}

# CV = DensityCrossValidation(estimator = LSKDEx,
#                             cvFolds = cvFolds,
#                             parameterGrid = paramGridLSx,
#                             n_jobs = len(cvFolds),
#                             randomSearch = True,
#                             nIter = 5) 

In [ ]:
# ps = ParameterSampler(param_distributions = paramGridLSx, n_iter = 4)

In [ ]:
# list(ps.__iter__())

In [ ]:
# CV.fit(X = XTrain, y = yTrain)

In [ ]:
# CV.cvResults

In [ ]:
#| hide

# import ipdb
# from lightgbm import LGBMRegressor

# from dddex.loadData import *
# from dddex.levelSetKDEx_univariate import LevelSetKDEx, BaseLSx
# from dddex.wSAA import RandomForestWSAA

In [ ]:
#| hide

# data, XTrain, yTrain, XTest, yTest = loadDataYaz(returnXY = True)

In [ ]:
#| hide

# LGBM = LGBMRegressor(boosting_type = 'gbdt',
#                      n_jobs = 1)

# LGBM.fit(X = XTrain, y = yTrain)

In [ ]:
#| hide

# LSKDEx = LevelSetKDEx(estimator = LGBM, binSize = 100, weightsByDistance = False)
# LSKDEx.fit(XTrain, yTrain)

# densities = LSKDEx.getWeights(XTest, outputType = 'onlyPositiveWeightsValues')

In [ ]:
#| hide

# dataTrain = data[data.label == 'train']

# cvFolds = groupedTimeSeriesSplit(data = dataTrain, 
#                                  kFolds = 3, 
#                                  testLength = 28, 
#                                  groupFeature = 'id', 
#                                  timeFeature = 'dayIndex')

In [ ]:
#| hide

# CV = QuantileCrossValidationLSx(estimatorLSx = LSKDEx,
#                                 cvFolds = cvFolds,
#                                 parameterGridLSx = {'binSize': [10, 100, 400],
#                                                    'weightsByDistance': [True, False]},
#                                 parameterGridEstimator = {'max_depth': [3, 4],
#                                                           'n_estimators': [150, 210]},
#                                 randomSearchEstimator = True,
#                                 nIterEstimator = 2,
#                                 random_state = 44,
#                                 probs = [0.01, 0.49, 0.5, 0.999],
#                                 refitPerProb = True)

# CV.fit(X = XTrain, y = yTrain)

# CV2 = QuantileCrossValidation(estimator = LSKDEx,
#                               cvFolds = cvFolds,
#                               parameterGrid = {'binSize': [10, 100, 400],
#                                                'weightsByDistance': [True, False]},
#                               randomSearch = False,
#                               nIter = 1,
#                               random_state = 4444,
#                               probs = [0.01, 0.49, 0.5, 0.999],
#                               refitPerProb = True)

# CV2.fit(X = XTrain, y = yTrain)

In [ ]:
#| hide

# RF = RandomForestWSAA()

# CV3 = QuantileCrossValidation(estimator = RF,
#                              cvFolds = cvFolds,
#                              parameterGrid = {'max_depth': [3, 4],
#                                               'n_estimators': [150, 210]},
#                              probs = [0.01, 0.49, 0.5, 0.999],
#                              refitPerProb = True)

# CV3.fit(X = XTrain, y = yTrain)

In [ ]:
#| hide

# CV4 = DensityCrossValidation(estimator = LSKDEx,
#                             cvFolds = cvFolds,
#                             parameterGrid = {'binSize': [10, 100, 400],
#                                              'weightsByDistance': [True, False]},
#                             randomSearch = True,
#                             nIter = 5,
#                             random_state = 44,
#                             p = 2)

# CV4.fit(XTrain, yTrain)

In [ ]:
#| hide

# CV5 = DensityCrossValidationLSx(estimatorLSx = LSKDEx,
#                                cvFolds = cvFolds,
#                                parameterGridLSx = {'binSize': [10, 100, 400],
#                                                   'weightsByDistance': [True, False]},
#                                parameterGridEstimator = {'max_depth': [3, 4],
#                                                          'n_estimators': [150, 210]},
#                                randomSearchEstimator = True,
#                                nIterEstimator = 2,
#                                random_state = 44,
#                                p = 2)

# CV5.fit(X = XTrain, y = yTrain)